In [ ]:
from google.colab import drive

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler

from scipy.interpolate import interp1d

In [ ]:
drive.mount('/content/drive', force_remount=True)
data_dir = "/content/drive/My Drive/Inzynierka/Dane/"

Mounted at /content/drive


In [ ]:
csv_train = data_dir+"Competition/train.csv"

In [ ]:
columns = ['X','Y','Z','Device']
df_train = pd.read_csv(csv_train, usecols=columns)

In [ ]:
def prepareForSequences(df, seq_length, n):
  device_unique = df.Device.unique()
  data = pd.DataFrame({'X': [], 'Y': [],'Z': [], 'Device': []})
  sequences_count = []
  for device in device_unique:
    index = df.loc[df['Device'] == device].index[0]
    data = pd.concat([data, df.iloc[index:index+n*seq_length]], ignore_index=True)
    sequences_count.append(n)
  return data, sequences_count

In [ ]:
def getDataFromFile(file_path):
  with open(file_path, 'r') as file:
    lines = file.readlines()

    my_data = []
    for line in lines:
        row = list(map(float, line.strip().split(',')))
        my_data.append(row)
    my_data = np.array(my_data)
    return my_data

In [ ]:
def prepareMyData (data):
  l = data.shape[0]
  length = (l//291)*291
  data = data[:length]
  data_seq = create_sequences(data, 291)
  return data_seq

In [ ]:
def create_sequences(data, seq_length):
    X = []
    for i in range (int(len(data) / seq_length)):
        X.append(data[i:i+seq_length])
    return np.array(X)

In [ ]:
def interpolate_data(data):
  n_samples_5 = data.shape[0]  # Liczba próbek
  indices_5 = np.arange(n_samples_5)  # Indeksy próbek [0, 1, ..., n_samples_5-1]
  indices_50 = np.linspace(0, n_samples_5 - 1, (n_samples_5 - 1) * 10 + 1)  # Indeksy dla 50 Hz

  # Interpolacja każdej kolumny osobno
  interpolated_data = np.zeros((len(indices_50), data.shape[1]))  # Przygotowanie tablicy wynikowej

  for i in range(data.shape[1]):  # Iteracja po wymiarach (kolumnach)
      interp_func = interp1d(indices_5, data[:, i], kind='quadratic')  # Interpolacja liniowa
      interpolated_data[:, i] = interp_func(indices_50)  # Wynik dla wymiaru i
  return interpolated_data

In [ ]:
seq_length = 30 # długość sekwencji przed
n = 10 #liczba sekwencji

data, sequences_count = prepareForSequences(df_train, seq_length, n)

In [ ]:
len(sequences_count)

387

In [ ]:
X = data[['X', 'Y', 'Z']].to_numpy()
y = data['Device'].to_numpy()
# my_X, my_y = prepare_my_data(my_data, seq_length)
# X = np.vstack((X, my_X))

X_seq= create_sequences(X, seq_length)
y_seq = y[::seq_length]
y_seq = np.zeros(len(sequences_count)*n)
# y_seq = np.hstack((y_seq, my_y))
print(X_seq.shape)
print(y_seq.shape)

(3870, 30, 3)
(3870,)


In [ ]:
X_seq_interpolated = []
for seq in X_seq:
  interpolated_data = interpolate_data(seq)
  X_seq_interpolated.append(interpolated_data)
X_seq_interpolated=np.array(X_seq_interpolated)

In [ ]:
my_data1 = getDataFromFile('/content/drive/MyDrive/Inzynierka/Dane/moje/iga3020241122123434.txt')
my_data2 = getDataFromFile('/content/drive/MyDrive/Inzynierka/Dane/moje/iga3020241122123506.txt')
my_data3 = getDataFromFile('/content/drive/MyDrive/Inzynierka/Dane/moje/iga3020241122123539.txt')

my_data = np.concatenate((my_data1, my_data2, my_data3))

In [ ]:
tata_data = getDataFromFile('/content/drive/MyDrive/Inzynierka/Dane/moje/Krzysztof 20241205171209.txt')
mama_data = getDataFromFile('/content/drive/MyDrive/Inzynierka/Dane/moje/Monika20241205172743.txt')
mil_data = getDataFromFile('/content/drive/MyDrive/Inzynierka/Dane/moje/Milosz20241205171024.txt')

In [ ]:
tata_seq = prepareMyData(tata_data)
mama_seq = prepareMyData(mama_data)
mil_seq = prepareMyData(mil_data)
my_seq = prepareMyData(my_data)

In [ ]:
y_seq= np.append(y_seq, np.zeros(15))
y_seq= np.append(y_seq, np.ones(15))

In [ ]:
X_seq_all = np.concatenate((X_seq_interpolated, tata_seq, mama_seq, mil_seq, my_seq))

In [ ]:
print(X_seq_all.shape)
print(y_seq.shape)

(3900, 291, 3)
(3900,)


In [ ]:
reshaped_data = X_seq_all.reshape(-1, 3)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(reshaped_data)
original_shape = X_seq_all.shape
X_seq_all = scaled_data.reshape(original_shape)

In [ ]:
np.save('/content/drive/MyDrive/Inzynierka/Dane/moje/X.npy', X_seq_all)

In [ ]:
np.save('/content/drive/MyDrive/Inzynierka/Dane/moje/y.npy', y_seq)